In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import csv

# Input and output file paths
input_file = 'logs/metal_performance_log.csv'
output_file = "logs/performance_log_normalised.csv"

# Open the input file for reading
with open(input_file, mode='r') as infile:
    reader = csv.reader(infile)
    rows = list(reader)

# Normalize rows
normalized_rows = []
for row in rows:
    # Ensure all rows have the same number of columns
    if len(row) < len(rows[0]):
        row.append("NA")  # Add "NA" if the last column is missing
    normalized_rows.append(row)

# Write the normalized rows to the output file
with open(output_file, mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(normalized_rows)

print(f"Normalized CSV file saved as '{output_file}'.")

Normalized CSV file saved as 'logs/performance_log_normalised.csv'.


In [ ]:
# Import data
df_other = pd.read_csv('Other-Implementations/logs/performance_log.csv')
df_metal = pd.read_csv('logs/metal_performance_log.csv')

# Combine both datasets into one DataFrame
df_other['Source'] = 'Other'
df_metal['Source'] = 'Metal'
df_combined = pd.concat([df_other, df_metal], ignore_index=True)

# Group by ExecutionType, NumIterations, and Source, and calculate mean for both columns
grouped = df_combined.groupby(['Source', 'ExecutionType', 'NumIterations'])[['ReconstructionTime_ms', 'FinalUpdateNorm']].mean().reset_index()

# Pivot the data to create a single DataFrame
pivoted = grouped.pivot(index='NumIterations', columns=['Source', 'ExecutionType'], values=['ReconstructionTime_ms', 'FinalUpdateNorm'])

# Flatten the multi-level columns for easier readability
pivoted.columns = ['_'.join(col).strip() for col in pivoted.columns.values]

pivoted = pivoted.rename(columns={
    'ReconstructionTime_ms_Other_Sequential': 'Sequential_Time',
    'ReconstructionTime_ms_Other_openmp': 'OpenMP_Time',
    'FinalUpdateNorm_Other_Sequential': 'Sequential_Norm',
    'FinalUpdateNorm_Other_openmp': 'OpenMP_Norm',
    'ReconstructionTime_ms_Metal_Metal': 'Metal_Time',
    'FinalUpdateNorm_Metal_Metal': 'Metal_Norm'
})

# Define the new column order
new_column_order = [
    'Sequential_Time',          # Move Metal_Time to column 1
    'OpenMP_Time',     # Move Sequential_Time to column 2
    'Metal_Time',         # Move OpenMP_Time to column 3
    'Sequential_Norm',     # Keep Sequential_Norm in column 4
    'OpenMP_Norm',         # Keep OpenMP_Norm in column 5
    'Metal_Norm'           # Keep Metal_Norm in column 6
]

# Reorder the columns
pivoted = pivoted[new_column_order]

# Add speedup columns
pivoted['Speedup_OMP_vs_Seq'] = pivoted['Sequential_Time'] / pivoted['OpenMP_Time']
pivoted['Speedup_Metal_vs_Seq'] = pivoted['Sequential_Time'] / pivoted['Metal_Time']

pivoted = pivoted.round(2)

styled_table = pivoted.style.format(precision=2)\
    .background_gradient(subset=['Speedup_Metal_vs_Seq'], cmap='coolwarm')

# Drop the norm columns for plotting
pivoted_plot = pivoted.drop(columns=['Sequential_Norm', 'OpenMP_Norm', 'Metal_Norm'])
pivoted_plot = pivoted_plot.reset_index()

styled_table = pivoted_plot.style.format(precision=2) \
    .background_gradient(subset=['Speedup_Metal_vs_Seq'], cmap='coolwarm')
styled_table

,NumIterations,Sequential_Time,OpenMP_Time,Metal_Time,Speedup_OMP_vs_Seq,Speedup_Metal_vs_Seq
0,10,2247.61,671.86,71.25,3.35,31.55
1,100,24452.89,7492.12,473.54,3.26,51.64
2,500,127594.17,36421.52,2185.59,3.50,58.38
3,1000,200628.20,75266.26,4309.20,2.67,46.56
4,2000,513818.00,163745.00,8963.58,3.14,57.32
